# Trabajo Práctico 2: Análisis con Voting - Organización de Datos

**Alumnos y Padrón**  
* Grassano, Bruno - 103855  
* Romero, Adrián   - 103371

https://github.com/brunograssano/TP-Organizacion-de-datos

## Importamos las bibiliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import tree

In [ ]:
from preprocessing import prepararSetDeDatos
from preprocessing import prepararSetDeHoldout
from preprocessing import prepararSetDeValidacion
from preprocessing import conversionAVariablesNumericasNormalizadas
from preprocessing import expansionDelDataset

In [ ]:
from funcionesAuxiliares import mostrarAUCScore
from funcionesAuxiliares import mostrarROCCurve
from funcionesAuxiliares import mostrarMatrizDeConfusion
from funcionesAuxiliares import escribirPrediccionesAArchivo
from funcionesAuxiliares import obtenerDatasets
from funcionesAuxiliares import obtenerHoldout

## Importamos los datos y los procesamos

In [ ]:
X,y = obtenerDatasets()
X = prepararSetDeDatos(X)
y = prepararSetDeValidacion(y)

## Voting

Este modelo es un ensamble que consiste en la unión de varios modelos, los cuales votaran de que clase es una instancia. En este caso, decidimos armar el ensamble utilizando los modelos que mejores resultados (segun la metrica AUC-ROC) nos dieron en los otros notebooks.
Los elegidos fueron:
 * Árbol de decisión
 * SVM
 * Random Forest
 * Regresion logística

Cada uno de estos modelos lo recreamos con los mejores hiperparámetros que se encontraron en su notebook.

Para el preprocesamiento probamos primero con el basico, el cual incluye a todas las columnas que vienen en el dataframe.

In [ ]:
X_voting = conversionAVariablesNumericasNormalizadas(X) 

Dividimos el set de datos en sets de training y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_voting, y, test_size=0.25, random_state=0)

Inicializamos los modelos que usara Voting, cada uno con sus mejores hiperparametros encontrados. En el caso de random forest se les redujo algo la profundidad.

In [ ]:
regresion_logistica = LogisticRegression(penalty = 'none', solver = "saga",max_iter = 5000)
random_forest = RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy',max_depth=7)
svm = SVC(C=200, kernel='rbf', gamma=0.1,probability=True)
arbol = tree.DecisionTreeClassifier(random_state=117, max_depth=4, criterion = 'gini')

Creamos el modelo y lo entrenamos.

In [ ]:
voting = VotingClassifier(estimators=[('lr', regresion_logistica), ('rf', random_forest),('svm',svm),('tree',arbol)], voting='soft')

In [ ]:
voting.fit(X_train, y_train)

#### Evaluación de métricas

Ahora si, realizamos las predicciones y observamos las métricas.

In [ ]:
y_pred = voting.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
mostrarMatrizDeConfusion(y_pred,y_test)

In [ ]:
mostrarROCCurve(voting,"Voting",X_test,X_train,y_test,y_train)

In [ ]:
mostrarAUCScore(voting,"Voting",X_test,y_test)

Observamos que obtuvimos una mejora en comparación a los otros modelos en varias de las métricas, por no decir todas.

#### Con otro preprocesamiento

Vemos ahora si podemos mejorar este resultado utilizando el Dataframe expandido.

In [ ]:
X = expansionDelDataset(X)

In [ ]:
X.head()

In [ ]:
columnas_codificables_extra = ['pago_categorizado','edades_estratificadas','categoria_invitados']
columnas_numericas_extra = ['2_clusters','4_clusters','10_clusters','cantidad_total_invitados','total_pagado']
X_voting_exp = conversionAVariablesNumericasNormalizadas(X,columnas_codificables_extra,columnas_numericas_extra) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_voting_exp, y, test_size=0.25, random_state=0)

In [ ]:
voting_exp = VotingClassifier(estimators=[('lr', regresion_logistica), ('rf', random_forest),('svm',svm),('tree',arbol)], voting='soft')

In [ ]:
voting_exp.fit(X_train, y_train)

#### Evaluamos

In [ ]:
y_pred = voting_exp.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
mostrarROCCurve(voting_exp,"Voting",X_test,X_train,y_test,y_train)

In [ ]:
mostrarAUCScore(voting_exp,"Voting",X_test,y_test)

Vemos que empeoro probando con toda la informacion nueva.

#### Otro preprocesamiento mas

Vemos si cambia algo sacar algunas columnas. Sacamos la del resultado con 10 clusters y la de la cantidad total de invitados.

In [ ]:
columnas_codificables_extra = ['pago_categorizado','edades_estratificadas','categoria_invitados']
columnas_numericas_extra = ['2_clusters','4_clusters','total_pagado']
X_voting_exp2 = conversionAVariablesNumericasNormalizadas(X,columnas_codificables_extra,columnas_numericas_extra) 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_voting_exp2, y, test_size=0.25, random_state=0)

In [ ]:
voting_exp2 = VotingClassifier(estimators=[('lr', regresion_logistica), ('rf', random_forest),('svm',svm),('tree',arbol)], voting='soft')

In [ ]:
voting_exp2.fit(X_train, y_train)

#### Evaluamos este preprocesamiento

In [ ]:
y_pred = voting_exp2.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=['No vuelve','Vuelve']))

In [ ]:
mostrarROCCurve(voting_exp2,"Voting",X_test,X_train,y_test,y_train)

In [ ]:
mostrarAUCScore(voting_exp2,"Voting",X_test,y_test)

Mejoro bastante respecto al anterior, pero no llego a superar la metrica del primero.

## Predicciones sobre el nuevo archivo

Realizamos ahora las predicciones del nuevo archivo entregado.

In [ ]:
holdout = obtenerHoldout()

holdout = prepararSetDeHoldout(holdout)
holdout_voting = conversionAVariablesNumericasNormalizadas(holdout)

Realizamos las predicciones y escribimos al archivo CSV.

In [ ]:
predicciones_holdout = voting.predict(holdout_voting)

In [ ]:
escribirPrediccionesAArchivo(predicciones_holdout,"Voting")